In [71]:
import pandas as pd
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [6]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies
collection = db.companies

In [ ]:
# FILTER {$and: [ {deadpooled_year:null}, {founded_year: {$gt:2003}}, {$or: [{category_code:'games_video'},{category_code:'software'}] }]}
# PROJECT {name:1 , category_code:1, number_of_employees:1, founded_year:1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
# SORT {number_of_employees:-1}

In [82]:
def query_to_db():
    cursor = collection.find({'$and': 
                        [ {'deadpooled_year':None}, 
                          {'founded_year': {'$gt':2003}}, 
                          {'$or': [{'category_code':'games_video'},
                                   {'category_code':'software'}] 
                        }]}
                        , {'name':1 , 'category_code':1, 'number_of_employees':1, 'founded_year':1, 
                           "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1}
                        ).sort('number_of_employees', -1)
    return pd.DataFrame(cursor)

In [83]:
# Main

df= query_to_db()
#df.offices[1][0]['city']

In [84]:
df.head()

,_id,category_code,founded_year,name,number_of_employees,offices
0,52cdef7f4bab8bd67529c4e1,software,2004,UCWeb,1700.0,"[{'city': 'Guangzhou', 'country_code': 'CHN', ..."
1,52cdef7e4bab8bd67529a77a,software,2006,Veeam Software,1500.0,"[{'city': 'Columbus', 'country_code': 'USA', '..."
2,52cdef7e4bab8bd67529aa46,software,2006,Brain Bakery,1000.0,"[{'city': 'Ringwood', 'country_code': 'GBR', '..."
3,52cdef7e4bab8bd67529acec,software,2006,Relax Solutions Pvt Ltd,900.0,"[{'city': 'Faisalabad', 'country_code': 'PAK',..."
4,52cdef7e4bab8bd67529a8b4,games_video,2004,GREE,700.0,"[{'city': 'Tokyo', 'country_code': 'JPN', 'lat..."
